In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('ap_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00007FF6FDA3EEB2+31554]
	(No symbol) [0x00007FF6FD9B7EE9]
	(No symbol) [0x00007FF6FD87872A]
	(No symbol) [0x00007FF6FD84D995]
	(No symbol) [0x00007FF6FD8F44D7]
	(No symbol) [0x00007FF6FD90C051]
	(No symbol) [0x00007FF6FD8ECDD3]
	(No symbol) [0x00007FF6FD8BA33B]
	(No symbol) [0x00007FF6FD8BAED1]
	GetHandleVerifier [0x00007FF6FDD48B2D+3217341]
	GetHandleVerifier [0x00007FF6FDD95AF3+3532675]
	GetHandleVerifier [0x00007FF6FDD8B0F0+3489152]
	GetHandleVerifier [0x00007FF6FDAEE786+750614]
	(No symbol) [0x00007FF6FD9C376F]
	(No symbol) [0x00007FF6FD9BEB24]
	(No symbol) [0x00007FF6FD9BECB2]
	(No symbol) [0x00007FF6FD9AE17F]
	BaseThreadInitThunk [0x00007FFAD5D0257D+29]
	RtlUserThreadStart [0x00007FFAD7B6AF28+40]

Error occurred 

Error occurred while accessing page 2: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00007FF6FDA3EEB2+31554]
	(No symbol) [0x00007FF6FD9B7EE9]
	(No symbol) [0x00007FF6FD87872A]
	(No symbol) [0x00007FF6FD84D995]
	(No symbol) [0x00007FF6FD8F44D7]
	(No symbol) [0x00007FF6FD90C051]
	(No symbol) [0x00007FF6FD8ECDD3]
	(No symbol) [0x00007FF6FD8BA33B]
	(No symbol) [0x00007FF6FD8BAED1]
	GetHandleVerifier [0x00007FF6FDD48B2D+3217341]
	GetHandleVerifier [0x00007FF6FDD95AF3+3532675]
	GetHandleVerifier [0x00007FF6FDD8B0F0+3489152]
	GetHandleVerifier [0x00007FF6FDAEE786+750614]
	(No symbol) [0x00007FF6FD9C376F]
	(No symbol) [0x00007FF6FD9BEB24]
	(No symbol) [0x00007FF6FD9BECB2]
	(No symbol) [0x00007FF6FD9AE17F]
	BaseThreadInitThunk [0x00007FFAD5D0257D+29]
	RtlUserThreadStart [0x00007FFAD7B6AF28+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/v

Error occurred while accessing page 4: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00007FF6FDA3EEB2+31554]
	(No symbol) [0x00007FF6FD9B7EE9]
	(No symbol) [0x00007FF6FD87872A]
	(No symbol) [0x00007FF6FD84D995]
	(No symbol) [0x00007FF6FD8F44D7]
	(No symbol) [0x00007FF6FD90C051]
	(No symbol) [0x00007FF6FD8ECDD3]
	(No symbol) [0x00007FF6FD8BA33B]
	(No symbol) [0x00007FF6FD8BAED1]
	GetHandleVerifier [0x00007FF6FDD48B2D+3217341]
	GetHandleVerifier [0x00007FF6FDD95AF3+3532675]
	GetHandleVerifier [0x00007FF6FDD8B0F0+3489152]
	GetHandleVerifier [0x00007FF6FDAEE786+750614]
	(No symbol) [0x00007FF6FD9C376F]
	(No symbol) [0x00007FF6FD9BEB24]
	(No symbol) [0x00007FF6FD9BECB2]
	(No symbol) [0x00007FF6FD9AE17F]
	BaseThreadInitThunk [0x00007FFAD5D0257D+29]
	RtlUserThreadStart [0x00007FFAD7B6AF28+40]

Error occurred while accessing page 5: Message: no such window: target window alr

NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bangalore to Chittoor (Andhra Pradesh),https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC - 9020,Express(Non AC Seater),17:30,04h 25m,21:55,3.4,INR 186,37 Seats available
1,Bangalore to Chittoor (Andhra Pradesh),https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC - 47211,Express(Non AC Seater),18:30,04h 25m,22:55,3.3,INR 186,42 Seats available
2,Bangalore to Chittoor (Andhra Pradesh),https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC - 5457,Super Luxury (Non AC Seater 2+2 Push Back),21:30,04h 30m,02:00,4.1,INR 277,25 Seats available
3,Bangalore to Chittoor (Andhra Pradesh),https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC - 47006,Express(Non AC Seater),21:30,05h 00m,02:30,4.2,INR 217,42 Seats available
4,Bangalore to Chittoor (Andhra Pradesh),https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC - 5449,Super Luxury (Non AC Seater 2+2 Push Back),21:30,05h 45m,03:15,3.5,INR 304,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
110,Kurnool to Hyderabad,https://www.redbus.in/bus-tickets/kurnool-to-h...,Northern Travels,Bharat Benz A/C Seater /Sleeper (2+1),21:30,02h 28m,23:58,2.9,INR 1499,6 Seats available
111,Kurnool to Hyderabad,https://www.redbus.in/bus-tickets/kurnool-to-h...,Morning Star Travels,Volvo Multi Axle B9R A/C Sleeper (2+1),23:59,03h 21m,03:20,2.9,INR 1190,28 Seats available
112,Kurnool to Hyderabad,https://www.redbus.in/bus-tickets/kurnool-to-h...,Geetha Bus Travels,NON A/C Sleeper (2+1),23:59,06h 01m,06:00,2.1,INR 800,12 Seats available
113,Kurnool to Hyderabad,https://www.redbus.in/bus-tickets/kurnool-to-h...,True Bus,Volvo Multi Axle B9R A/C Sleeper (2+1),23:59,05h 01m,05:00,0,INR 4400,36 Seats available
